# Data Mining Course Spark Exercise
## Sharif University of Technology

# Please enter your name below:
# Name: Ilia Hashemi Rad
# Student Number: 99102456

## Install Pypark & Initialization
Uncomment this section if you use google colab or local pc

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("HW3") \
    .master("local[*]") \
    .getOrCreate()

sc=spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/01 21:16:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Reading the data

In [2]:
tweets_rdd = sc.textFile("twitter_data_v2.jsonl")

# Showing Data Hierarchical

### Tweet type : Retweeted

In [19]:
import json

def create_tree(tweet_data):
    tree = {}
    
    # Extracting top-level fields
    tree["id"] = tweet_data["id"]
    tree["user"] = {
        "id": tweet_data["user"]["id"],
        "screen_name": tweet_data["user"]["screen_name"],
        "name": tweet_data["user"]["name"],
        "description": tweet_data["user"]["description"]
    }
    tree["tweet_type"] = tweet_data["tweet_type"]
    tree["in_reply_to_status_id_str"] = tweet_data["in_reply_to_status_id_str"]
    tree["in_reply_to_user_id_str"] = tweet_data["in_reply_to_user_id_str"]
    
    # Quoted status
    quoted_status = tweet_data.get("quoted_status")
    if quoted_status:
        tree["quoted_status"] = {
            "id": quoted_status["id"],
        }
    
    # Retweeted status
    retweeted_status = tweet_data.get("retweeted_status")
    if retweeted_status:
        tree["retweeted_status"] = {
            "id": retweeted_status["id"],
            "created_at": retweeted_status["created_at"],
            "user": {
                "id": retweeted_status["user"]["id"],
                "screen_name": retweeted_status["user"]["screen_name"],
                "name": retweeted_status["user"]["name"],
                "description": retweeted_status["user"]["description"]
            }
        }
    
    return tree

# Example for retweeted data
sample_data = json.loads(tweets_rdd.take(10)[0])
tweet_tree = create_tree(sample_data)
print(json.dumps(tweet_tree, indent=2, ensure_ascii=False))


{
  "id": "1736536753815834746",
  "user": {
    "id": "2808957837",
    "screen_name": "maryaa_maryam",
    "name": "maryam jafari",
    "description": ""
  },
  "tweet_type": "retweeted",
  "in_reply_to_status_id_str": null,
  "in_reply_to_user_id_str": null,
  "retweeted_status": {
    "id": "1736379038376120479",
    "created_at": 1702819997,
    "user": {
      "id": "2960042892",
      "screen_name": "Ivar_lathbrug2",
      "name": "آیوار ۲",
      "description": "باورمند به نظام پادشاهی مشروطه با @PahlaviReza #جاویدشاه @PhoenixprjIran     https://t.co/kjMxf53cIQ"
    }
  }
}


### Tweet type : Replied

In [11]:
# Example for replied data
sample_data = json.loads(tweets_rdd.take(10)[3])
tweet_tree = create_tree(sample_data)
print(json.dumps(tweet_tree, indent=2, ensure_ascii=False))

{
  "id": "1736536759964512322",
  "user": {
    "id": "1406742149048287242",
    "screen_name": "sun_saeid",
    "name": "حق نامه (تلخ)",
    "description": "از زندگی بگیم ولی واقعیتشو"
  },
  "tweet_type": "replied",
  "in_reply_to_status_id_str": "1736356968162378084",
  "in_reply_to_user_id_str": "1643210821068046338"
}


### Tweet type : Generated

In [13]:
# Example for generated data
sample_data = json.loads(tweets_rdd.take(10)[1])
tweet_tree = create_tree(sample_data)
print(json.dumps(tweet_tree, indent=2, ensure_ascii=False))

{
  "id": "1736536756835512564",
  "user": {
    "id": "22495731",
    "screen_name": "AlArabiya_Fa",
    "name": "العربیه فارسی",
    "description": "«العربيه فارسى» به عنوان بخشی از شبكه «العربيه» در سال 2008 راه‌اندازی شد. \nایمیل تحریریه العربیه فارسی:\neditor.farsi@alarabiya.net\nhttps://t.co/kAD6bIfOgv"
  },
  "tweet_type": "generated",
  "in_reply_to_status_id_str": null,
  "in_reply_to_user_id_str": null
}


### Tweet type : Quoted

In [20]:
# Example for quoted data
sample_data = json.loads(tweets_rdd.take(10)[7])
tweet_tree = create_tree(sample_data)
print(json.dumps(tweet_tree, indent=2, ensure_ascii=False))

{
  "id": "1736536759943450888",
  "user": {
    "id": "1655373597001523200",
    "screen_name": "ScarFace2582",
    "name": "𝐒𝐂𝐀𝐑𝐅𝐀𝐂𝐄 ♔",
    "description": "♧\n#ایران_معبد_ماست #پاینده_ایران #جاویدشاه\n♧"
  },
  "tweet_type": "quoted",
  "in_reply_to_status_id_str": null,
  "in_reply_to_user_id_str": null,
  "quoted_status": {
    "id": "1736113075911077948"
  },
  "retweeted_status": {
    "id": "1736536140063109215",
    "created_at": 1702857453,
    "user": {
      "id": "893757772025798656",
      "screen_name": "YaarDabestaani",
      "name": "𓄂✺Yaar - يار ‌‌دبستانی 🏳️‍🌈",
      "description": "Human rights activist, 🌈LGBTQ🏳️‍🌈, animal lover🐕‍🦺🐈‍⬛🦥, Atheist, 👩🏿‍💼=🧑🏻‍💼=👩🏼‍💼=🧑🏿‍💼 #MahsaAmini,  My inner child: @bri30s"
    }
  }
}


# Determining a Dictionary of Reference Tweet of Quoted Tweets

In [3]:
import json

tweets_rdd = sc.textFile("twitter_data_v2.jsonl")

# Parse JSON strings to Python dictionaries
tweets_json = tweets_rdd.map(json.loads)
del tweets_rdd

# Create an RDD of quoted tweet ids
quoted_tweets_ids = tweets_json.filter(lambda x: x['tweet_type'] == "quoted").map(lambda x: (x.get('quoted_status').get('id'), None))

In [4]:
# Join the original RDD with the quoted tweet ids RDD
quoted_rdd = tweets_json.map(lambda x: (x['id'], x.get('user').get('id'))).join(quoted_tweets_ids)

# Remove the None values from the result
quoted_list = quoted_rdd.map(lambda x: (x[0], x[1][0])).collect()

# Create a dictionary of tweets which are the reference of quoted tweets as (tweet ID, user ID)
quoted_dict = dict(quoted_list)

In [5]:
print(dict(quoted_list[:20]))

{'1735116408798027896': '1576297375970525184', '1734416486838493585': '1017869466103877634', '1735127269465862373': '568002438', '1735519944362586556': '1128549743855112192', '1736630273771491555': '1062336497737699328', '1734085506948894902': '1577551087887851520'}


# Determining Users' Degree of Relation

In [10]:
import json

tweets_rdd = sc.textFile("twitter_data_v2.jsonl")

# Parse JSON strings to Python dictionaries
tweets_json = tweets_rdd.map(json.loads)
del tweets_rdd

# Filter out generated tweets
relevant_tweets = tweets_json.filter(lambda x: x['tweet_type'] in ["retweeted", "replied", "quoted"])

# Extract users' interaction rate with each other output 
def extract_tweet_rates(tweet):
    user_id = tweet.get("user", {}).get("id", None)
    tweet_type = tweet.get("tweet_type", None) 
    if tweet_type == 'replied':
        user_ref = tweet.get("in_reply_to_user_id_str", None) # reference user
        return (user_id, user_ref, 1)
    elif tweet_type == 'retweeted':
        user_ref = tweet.get("retweeted_status", {}).get("user", None).get("id", None) # reference user
        return (user_id, user_ref, 2)
    else:
        quoted_tweet_id = tweet.get("quoted_status", {}).get("id", None) # ID of tweet quoted from
        try:
            user_ref = quoted_dict[quoted_tweet_id] # user of tweet quoted from
            return (user_id, user_ref, 3)
        except:
            return None # return none if the reference tweet not found in the data (so we can't find the reference user)

user_rating = relevant_tweets.map(extract_tweet_rates).filter(lambda x: x!= None)

user_rating.takeSample(False, 8, 42)

[('1696062547533684736', '1351371042', 1),
 ('1012004974690275329', '3189754680', 2),
 ('2583526102', '1219846015345483779', 1),
 ('1561681142344044545', '1675560941885222913', 1),
 ('1655986795224367127', '1184763184948957185', 1),
 ('1614593362123710466', '1155366665019908096', 2),
 ('1327700517796114434', '820372916722434048', 2),
 ('1576699553948090368', '1638951842271862791', 1)]

# Finding Similar Users

## Ordinary Random Walk (Haven't been used because of more FP as explained on report)

In [ ]:
target_user_id = '165605880'


graph = user_rating.map(lambda x: (x[0], (x[1], x[2]))).groupByKey().mapValues(list)

# Define a transition probability function
# In our case, we use the edge weight as the probability
def transition_probability(user, neighbors):
    # Normalize the edge weights to sum up to 1
    total_weight = sum([x[1] for x in neighbors])
    normalized_weights = [x[1] / total_weight for x in neighbors]
    # Choose a random neighbor based on the normalized weights
    import random
    chosen_neighbor = random.choices(neighbors, weights=normalized_weights, k=1)[0]
    return chosen_neighbor

# Run the Random Walk algorithm on the graph
# In our case, we start from the target user and follow the transition probability function for 10 steps
# We record the nodes visited by the agent and their similarity scores
# The similarity score is the sum of the edge weights along the path
def random_walk(graph, start, steps):
    # Initialize the current node, the path, and the similarity scores
    current = start
    path = [current]
    similarity_scores = {}
    # Repeat for the given number of steps
    for _ in range(steps):

        try:
        # Get the neighbors of the current node from the graph
            neighbors = graph.filter(lambda kv: kv[0] == current).values().collect()[0]
        except:
            break
        # If there are no neighbors, stop the walk
        if len(neighbors) == 0:
            break
        # Otherwise, choose a neighbor based on the transition probability function
        chosen_neighbor = transition_probability(current, neighbors)
        # Update the current node, the path, and the similarity scores
        current = chosen_neighbor[0]
        path.append(current)
        similarity_scores[current] = similarity_scores.get(current, 0) + chosen_neighbor[1]
    # Return the path and the similarity scores
    return (path, similarity_scores)

# Call the random_walk function on the graph with the target user and 10 steps
path, similarity_scores = random_walk(graph, target_user_id, 50)

# Return the list of similar users and their similarity scores, sorted by descending order of similarity
similar_users = list(dict(sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)).keys())
print(similar_users)


## Random Walk with Restart

### Determining the desired Graph

In [11]:
# Create a graph from the user_rating RDD
graph = user_rating.flatMap(lambda x: [(x[0], (x[1], x[2])), (x[1], (x[0], x[2]))]) \
                   .groupByKey() \
                   .mapValues(list) \
                   .collectAsMap()

print({v:graph[v] for v in [list(graph.keys())[k] for k in range(1)]})

{'933731737686994946': [('1287439194663653376', 1), ('45284178', 1), ('1457064158025879554', 1), ('1458463785300148229', 1), ('1596756634818207746', 3), ('495082191', 1), ('495082191', 1), ('430566432', 1), ('1038187447283343361', 1), ('1275069380569657344', 1), ('1451085166970118146', 1), ('1314403966030090242', 1), ('1222946740929617920', 1), ('1550915415110549505', 1), ('1038187447283343361', 1), ('1451085166970118146', 1), ('1038187447283343361', 1), ('1595111714143572000', 1), ('1038187447283343361', 1), ('1038187447283343361', 1), ('1355183936641576965', 1), ('880414261478125569', 1), ('1046105731936776194', 1), ('1296386200497000454', 1), ('1177099107539791872', 1), ('1671085861532753924', 1), ('1671085861532753924', 1), ('1328753786748362759', 1), ('1595111714143572000', 1), ('1296386200497000454', 1), ('1296386200497000454', 1), ('1296386200497000454', 1), ('1444240663143399424', 1), ('1595111714143572000', 1), ('1005696819865808897', 3), ('1595111714143572000', 1), ('10861677

### Applying RWR algorithm on the Graph

In [12]:
import random

# Define a function to perform one step of random walk
def random_walk_step(node, graph):
    # Get the neighbors and weights of the current node
    neighbors, weights = zip(*graph[node])
    # Randomly choose a neighbor based on the weights
    next_node = random.choices(neighbors, weights=weights)[0]
    # Return the next node
    return next_node

# Define a function to perform multiple steps of random walk with restart
def random_walk_with_restart(node, graph, steps, restart_prob):
    # Initialize a dictionary to store the visit counts of the nodes
    visit_count = {n: 0 for n in graph}
    # Set the current node to the target node
    current_node = node
    # Loop for the given number of steps
    for i in range(steps):
        # Update the visit count of the current node
        visit_count[current_node] += 1
        # Generate a random number between 0 and 1
        r = random.random()
        # If the random number is less than the restart probability, restart the random walk from the target node
        if r < restart_prob:
            current_node = node
        # Otherwise, perform one step of random walk
        else:
            current_node = random_walk_step(current_node, graph)
    # Return the visit count dictionary
    return visit_count

# Define a function to find the top-k similar users to a given user
def find_similar_users(node, graph, steps, restart_prob, k):
    # Perform random walk with restart from the given node
    visit_count = random_walk_with_restart(node, graph, steps, restart_prob)
    # Sort the nodes by their visit counts in descending order
    sorted_nodes = sorted(visit_count, key=visit_count.get, reverse=True)
    # Return the first k nodes, excluding the target node
    return sorted_nodes[1:k+1]

# Set the target user id
target_user_id = '165605880'
# Set the number of steps, restart probability, and k
steps = 10000
restart_prob = 0.05
k = 50
# Find the top-k similar users to the target user
similar_users = find_similar_users(target_user_id, graph, steps, restart_prob, k)
# Print the result
print(f"The top-{k} similar users to user {target_user_id} are: {similar_users}")


The top-50 similar users to user 165605880 are: ['931938860963061760', '2221999644', '852104627566899200', '1640268020374294529', '3390890626', '1555561052', '1624682333394595841', '1430212798567563264', '1193634420000993280', '1096520546987184130', '947844496225644545', '1300119897737961474', '806048003672985601', '2184707526', '82311780', '1269647773307199488', '69058391', '1094620930557517832', '1717719501784170496', '1702354885063192579', '870292251389382657', '1706920934383276032', '1702217750054162432', '1678765614700318721', '845381446516822016', '3554230816', '988405852037156864', '865329625810980865', '1551110542412087297', '971328533275512832', '1617465622111129600', '848911278324273153', '2723935129', '1656228998228189184', '1276187974070403073', '841396621698985984', '2318855006', '1379875551356260361', '1666823496054501376', '2919269142', '1691082595549806592', '1643280415401099264', '868736088755453953', '1607679663060525056', '1695321628912025601', '2205932795', '1162722

# Evaluation

### Recommend Some Tweets based on Similar Users

In [13]:
top_recommended_tweets = tweets_json.filter(lambda x: x.get('user').get('id') in similar_users).map(lambda x: (x.get('user').get('id'), x['text'])).collect()

for tweet in top_recommended_tweets[:50]:
    print(f'ID of the similar user: {tweet[0]}\n')
    print('Tweet Text:')
    print(tweet[1])
    print('\n\n')

ID of the similar user: 786738726466707460

Tweet Text:
سازمان ملل از #رييسي_جلاد۶۷ استقبال می کند!
سازمان ملل جلاد #قتل_عام۶۷ را به مجمع جهانی پناهندگان دعوت می کند!
سازمان ملل حمله جوانان ستمزده بلوچ به نیروهای جنایتکار پاسگاه انتظامی راسک را محکوم می کند!!
ای تف بر این سازمان ملل و تف بر این سیاست جانبداری از قاتلان مردم ایران https://t.co/JUu2tNZqCN



ID of the similar user: 786738726466707460

Tweet Text:
لغو سفر رئیسی به #ژنو  نشان داد
دامنه و عمق داخلی و بین‌المللی جنبش دادخواهی #مردم_ایران بسابسا قدرت‌مندتر از زرادخانه‌ٔ دیکتاتوری آخوندی است
بله ما همدیگه رو داریم
#مرگ_بر_ستمگر_چه_شاه_باشه_چه_رهبر #نه_به_مماشات https://t.co/k5TJAe3mFn



ID of the similar user: 786738726466707460

Tweet Text:
🔴 یادداشت لیلا حسین‌زاد به‌مناسبت روز تولد زندانی سیاسی قهرمان #مریم_اکبری_منفرد 

▪️لیلا: به مریم فکر کردن، به‌خاطر خودمه؛ به این خاطر که وسط انبوه ابتذال و نیهیلیسم و کین‌توزی و بی‌رحمی و رقابت برای پسوندهای خنده‌دار، یادم به ‌یاد کسی هم هست که به ایمانی ایستاده، شاداب و خندان و در تقلا